#Sentiment Analysis on Product Reviews

--------------

<font color=teal size=5>Project Overview


This Jupyter notebook implements a comprehensive sentiment analysis on product reviews, following the project requirements outlined in the provided context. We will preprocess the data, perform sentiment scoring, conduct exploratory data analysis (EDA), create visualizations, and optionally build a simple web app component.

<font color=teal size=5>Dataset Description:

- Source: Amazon product reviews (simulated or loaded from a CSV file).
- Shape: 194,439 rows, 9 columns.
- Columns: reviewerID, asin, reviewerName, helpful, reviewText, overall, summary, unixReviewTime, reviewTime.
- Key fields for analysis: reviewText (text), overall (rating 1-5).

<font color=teal size=5>Tools and Libraries:

- Python 3.x
- Libraries: pandas, numpy, nltk, TextBlob, matplotlib, seaborn, wordcloud.
- Optional: Streamlit for web app.


------------
<font color=teal size=6>IMPORT LIBRARIES

--------------

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df=pd.read_json('/content/Cell_Phones_and_Accessories_5.json',lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194439 entries, 0 to 194438
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   reviewerID      194439 non-null  object
 1   asin            194439 non-null  object
 2   reviewerName    190920 non-null  object
 3   helpful         194439 non-null  object
 4   reviewText      194439 non-null  object
 5   overall         194439 non-null  int64 
 6   summary         194439 non-null  object
 7   unixReviewTime  194439 non-null  int64 
 8   reviewTime      194439 non-null  object
dtypes: int64(2), object(7)
memory usage: 13.4+ MB
